<a href="https://colab.research.google.com/github/toandaominh1997/Kaggle/blob/master/Elo_Merchant_Category_Recommendation/Elo_Merchant_Category_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

import zipfile
with zipfile.ZipFile('drive/My Drive/Datasets/Elo Merchant Category Recommendation/Elo Merchant Category Recommendation.zip', 'r') as cin:
  cin.extractall('input/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import time
import sys
import datetime

# Import Library
def read_data(filename):
  df = pd.read_csv(filename)
  return df
def read_transactions(filename):
  df = pd.read_csv(filename, parse_dates=['purchase_date'])
  return df

def label_binarize(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df
def onehotencoder(df):
  df = pd.get_dummies(df, columns=['category_2', 'category_3'])
  return df
train = read_data('input/train.csv')
test = read_data('input/test.csv')

his_transactions = read_transactions('input/historical_transactions.csv')
new_transactions = read_transactions('input/new_merchant_transactions.csv')
#print(his_transactions.columns)

his_transactions = label_binarize(his_transactions)
new_transactions = label_binarize(new_transactions)
his_transactions = onehotencoder(his_transactions)
new_transactions = onehotencoder(new_transactions)

auth_transactions = his_transactions[his_transactions['authorized_flag']==1]
his_transactions = his_transactions[his_transactions['authorized_flag']==0]


In [0]:
his_transactions['purchase_month'] = his_transactions['purchase_date'].dt.month
auth_transactions['purchase_month'] = auth_transactions['purchase_date'].dt.month
new_transactions['purchase_month'] = new_transactions['purchase_date'].dt.month

def aggregate_transactions(history):
  history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).\
                                      astype(np.int64) * 1e-9
  agg_func = {
        'category_1': ['sum', 'mean'],
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        'merchant_id': ['nunique'],
        'merchant_category_id': ['nunique'],
        'state_id': ['nunique'],
        'city_id': ['nunique'],
        'subsector_id': ['nunique'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_month': ['mean', 'max', 'min', 'std'],
        'purchase_date': [np.ptp, 'min', 'max'],
        'month_lag': ['min', 'max']
        }
    
  agg_history = history.groupby(['card_id']).agg(agg_func)
  agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
  agg_history.reset_index(inplace=True)
    
  df = (history.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
  agg_history = pd.merge(df, agg_history, on='card_id', how='left')

  return agg_history
his = aggregate_transactions(his_transactions)
his.columns = ['his'+c if c!='card_id' else c for c in his.columns]
new = aggregate_transactions(new_transactions)
new.columns = ['new'+c if c!='card_id' else c for c in his.columns]
auth = aggregate_transactions(auth_transactions)
auth.columns = ['auth'+c if c!='card_id' else c for c in his.columns]
def aggregate_per_month(history):
    grouped = history.groupby(['card_id', 'month_lag'])

    agg_func = {
            'purchase_amount': ['count', 'sum', 'mean', 'min', 'max', 'std'],
            'installments': ['count', 'sum', 'mean', 'min', 'max', 'std'],
            }

    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = ['_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)

    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std'])
    final_group.columns = ['_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True)
    
    return final_group
final = aggregate_per_month(his_transactions)

del his_transactions
del new_transactions
del auth_transactions

In [0]:
def mergewithdata(X, y):
  df = pd.merge(X, y, on='card_id', how='left')
  return df
train = mergewithdata(train, his)
train = mergewithdata(train, new)
train = mergewithdata(train, auth)

test = mergewithdata(test, his)
test = mergewithdata(test, new)
test = mergewithdata(test, auth)
del his
del new
del auth

In [5]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,histransactions_count,hiscategory_1_sum,hiscategory_1_mean,hiscategory_2_1.0_mean,...,authhisinstallments_std,authhispurchase_month_mean,authhispurchase_month_max,authhispurchase_month_min,authhispurchase_month_std,authhispurchase_date_ptp,authhispurchase_date_min,authhispurchase_date_max,authhismonth_lag_min,authhismonth_lag_max
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,13.0,0.0,0.000000,1.000000,...,0.000000,7.979757,12,1,3.528570,20977987.0,1.498573e+09,1.519551e+09,-8,0
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,11.0,2.0,0.181818,0.818182,...,1.350634,6.144543,12,1,3.859177,33717687.0,1.483720e+09,1.517438e+09,-12,0
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,2.0,0.0,0.000000,0.000000,...,0.000000,4.634146,12,1,3.329836,35635623.0,1.484123e+09,1.519759e+09,-13,0
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,NaN,NaN,NaN,NaN,...,0.588974,7.740260,12,1,3.904797,13375339.0,1.506443e+09,1.519818e+09,-5,0
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,5.0,3.0,0.600000,0.000000,...,1.003929,5.554688,12,1,5.041261,9405641.0,1.510445e+09,1.519850e+09,-3,0


In [12]:
test_id = test['card_id']
X_train = train.dropna(axis=0)
X_test = test.drop(columns=['card_id', 'first_active_month'])
target = X_train['target']
X_train = X_train.drop(columns=['card_id', 'first_active_month', 'target'])
from sklearn.model_selection import train_test_split

#X_train, X_val, y_train, y_val = train_test_split(X_train, target, test_size=0.1, random_state=42)

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X_train, target)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [0]:
X_test = X_test.fillna(0)
submit = pd.DataFrame({'card_id':test['card_id'].values})
submit['target'] = rf.predict(X_test)
submit.to_csv('drive/My Drive/sub.csv', index = False)